In [3]:
import pandas as pd
import numpy as np
import spacy
import gensim
import string
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from wordcloud import STOPWORDS
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score

In [4]:
# import nltk
# nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/naresh/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
def custom_tokenizer(text):
    tokens = word_tokenize(text)
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    return words

In [5]:
train_full_df = pd.read_csv('train.csv')
test_full_df = pd.read_csv('test.csv')

In [6]:
labels = list(train_full_df.topic.unique())
labels.sort()

In [7]:
train_full_df['text_data'] = train_full_df["Review Title"] + '. ' + train_full_df["Review Text"]
train_full_df['text_data'] = train_full_df['text_data'].str.replace(r'<.*?>', '').str.lower()
test_full_df['text_data'] = test_full_df["Review Title"] + '. ' + test_full_df["Review Text"]
test_full_df['text_data'] = test_full_df['text_data'].str.replace(r'<.*?>', '').str.lower()

In [87]:
# train_full_df['topic'] = train_full_df['topic'].apply(lambda x: labels.index(x))
# train_full_df.to_csv('train_bert.csv', index=False)
# test_full_df.to_csv('test_bert.csv', index=False)

In [6]:
train_full_df = train_full_df.join(pd.get_dummies(train_full_df.topic.tolist()))

In [8]:
# train_df, test_df = train_test_split(train_full_df, test_size = 0.2, stratify=train_full_df.iloc[:, 2])

In [8]:
tfidf_vector = TfidfVectorizer(tokenizer=custom_tokenizer, ngram_range=(1,2), max_df=0.8, min_df=3, strip_accents='unicode', sublinear_tf=True)

In [8]:
tfidf_vector.fit(train_full_df["text_data"].tolist())
train_tfidf = tfidf_vector.transform(train_full_df["text_data"])
test_tfidf = tfidf_vector.transform(test_full_df["text_data"])

/anaconda3/envs/deepLearning/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aren', 'br', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'let', 'll', 'mustn', 're', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [9]:
# tfidf_vector.fit(train_df["text_data"].tolist())
# train_tfidf = tfidf_vector.transform(train_df["text_data"])
# test_tfidf = tfidf_vector.transform(test_df["text_data"])

In [10]:
train_tfidf,  test_tfidf

(<4767x15897 sparse matrix of type '<class 'numpy.float64'>'
 	with 163568 stored elements in Compressed Sparse Row format>,
 <1192x15897 sparse matrix of type '<class 'numpy.float64'>'
 	with 34572 stored elements in Compressed Sparse Row format>)

In [18]:
x = train_tfidf
test_x = test_tfidf

def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

def get_mdl(y):
    y = y.values
    r = np.log(pr(1,y) / pr(0,y))
    m = LogisticRegression(C=0.3, dual=True)
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

In [17]:
# actual = test_df.topic.tolist()
# preds = np.zeros((len(test_df), len(labels)))
# for i, j in enumerate(labels):
#     m,r = get_mdl(train_df[j])
#     preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]
# preds = np.argmax(preds, axis=1)
# preds = [labels[i] for i in preds]
# print(f"accuracy for = {accuracy_score(actual, preds)}")

/anaconda3/envs/deepLearning/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


accuracy for = 0.5461409395973155


In [19]:
preds = np.zeros((len(test_full_df), len(labels)))
for i, j in enumerate(labels):
    m,r = get_mdl(train_full_df[j])
    preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]
preds = np.argmax(preds, axis=1)
preds = [labels[i] for i in preds]

/anaconda3/envs/deepLearning/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [23]:
test_full_df['topic'] = preds
test_full_df[["Review Text", "Review Title", "topic"]].to_csv('submission.csv', index=False)